<a href="https://colab.research.google.com/github/Hemal-Mamtora/SE_project_sentiment/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()


In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
#4/_ACxkRkA0SxfLP297b55YsrXmNZriss_RBv3DEYk-Rl11wUM89BfJUg

In [0]:
# Download the file we just uploaded.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'target_file_id'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId='1rZsp75ohwtYkFHw6shdXOLX3rt-ctnXF')
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)

reviews_test = []
for line in downloaded:
  reviews_test.append(line.strip().decode('utf-8'))

request = drive_service.files().get_media(fileId='1-AFdTbeEDnk90-klK-4jqLYPjPTrFpKH')
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()
  
downloaded.seek(0)
  
reviews_train = [] # response.read().decode('utf-8').
for line in downloaded:
  reviews_train.append(line.strip().decode('utf-8'))

In [0]:
import re

REPLACE_NO_SPACE = re.compile(
    "(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
  reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
  reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

  return reviews


reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(
    binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(reviews_train_clean)
X = ngram_vectorizer.transform(reviews_train_clean)
X_test = ngram_vectorizer.transform(reviews_test_clean)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size=0.75
)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# target = [1 if i < 12500 else 0 for i in range(25000)]

# stop_words = ['in', 'of', 'at', 'a', 'the']
# tfidf_vectorizer = TfidfVectorizer(
#     binary=True, ngram_range=(1, 3), stop_words=stop_words)
# tfidf_vectorizer.fit(reviews_train_clean)
# X = tfidf_vectorizer.transform(reviews_train_clean)
# X_test = tfidf_vectorizer.transform(reviews_test_clean)

# X_train, X_val, y_train, y_val = train_test_split(
#     X, target, train_size=0.75
# )

In [0]:
#from sklearn.svm import SVC
#model = SVC(kernel='linear',C=0.01,  probability=True)
#model.fit(X, target)
#print("Final Accuracy: %s"
#      % accuracy_score(target, loaded_model.predict(X_test))) // Runs infinitely, maybe it does not converge

In [0]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=3, random_state=0)
clf.fit(X, target)
print("Final Accuracy: %s"
     % accuracy_score(target, clf.predict(X_test)))

y_pred_proba = clf.predict_proba(X_test)
for i, j in zip(y_pred_proba, target):
  print(i, j)

#Lemmatization did not improve the score(in fact it remained same) : randomforest, countvectorizer, 100, 2, 0
#SVC linear kernel with probability, runs infinitely, maybe it does not converge
#Final Accuracy: 0.72564 100, 2, 0 #randomforest, countvectorizer
#Final Accuracy: 0.79924 500, 2, 0 #randomforest, countvectorizer
#Final Aeccuracy: 0.81924 500. 3, 0 #randomforest with CountVectorizer
#Final Accuracy: 0.80388 500, 3, 0, using tfidf and RandomForestClassifier 
#Final Accuracy: 0.80548 # took too much time to run #xgboost classifier with TfidfVectorizer


Final Accuracy: 0.81924
[0.49581886 0.50418114] 1
[0.49348763 0.50651237] 1
[0.49307789 0.50692211] 1
[0.49434779 0.50565221] 1
[0.49319782 0.50680218] 1
[0.4934453 0.5065547] 1
[0.49622456 0.50377544] 1
[0.49499917 0.50500083] 1
[0.49271712 0.50728288] 1
[0.49520961 0.50479039] 1
[0.49663288 0.50336712] 1
[0.49574827 0.50425173] 1
[0.49501375 0.50498625] 1
[0.4966404 0.5033596] 1
[0.49111136 0.50888864] 1
[0.490888 0.509112] 1
[0.4977333 0.5022667] 1
[0.49451486 0.50548514] 1
[0.4960328 0.5039672] 1
[0.49577329 0.50422671] 1
[0.49542756 0.50457244] 1
[0.49671164 0.50328836] 1
[0.49807107 0.50192893] 1
[0.4974631 0.5025369] 1
[0.49442927 0.50557073] 1
[0.49488366 0.50511634] 1
[0.49394758 0.50605242] 1
[0.49904097 0.50095903] 1
[0.49907541 0.50092459] 1
[0.49475854 0.50524146] 1
[0.48747599 0.51252401] 1
[0.49934411 0.50065589] 1
[0.49233138 0.50766862] 1
[0.49574809 0.50425191] 1
[0.49574827 0.50425173] 1
[0.49561611 0.50438389] 1
[0.49664645 0.50335355] 1
[0.49235526 0.50764474] 1
[0

In [0]:
 #import nltk
 #nltk.download('wordnet')



In [0]:
#def get_lemmatized_text(corpus):
#    from nltk.stem import WordNetLemmatizer
#    lemmatizer = WordNetLemmatizer()
#    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]



In [0]:
#lemmatized_reviews_train = get_lemmatized_text(reviews_train_clean)
#lemmatized_reviews_test = get_lemmatized_text(reviews_test_clean)


In [0]:
#gram_vectorizer = CountVectorizer(
#    binary=True, ngram_range=(1, 3), stop_words=stop_words)
#ngram_vectorizer.fit(reviews_train_clean)
#X = ngram_vectorizer.transform(lemmatized_reviews_train)
#X_test = ngram_vectorizer.transform(lemmatized_reviews_test)

In [0]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#clf.fit(X, target)
#print("Final Accuracy: %s"
#     % accuracy_score(target, clf.predict(X_test)))
#
#y_pred_proba = clf.predict_proba(X_test)
#for i, j in zip(y_pred_proba, target):
#  print(i, j)
  
#Final Accuracy: 0.72564

In [0]:
#from xgboost import XGBClassifier
#model = XGBClassifier()
#model.fit(X, target)
#print("Final Accuracy: %s"
#     % accuracy_score(target, model.predict(X_test)))

#y_pred_proba = model.predict_proba(X_test)
#for i, j in zip(y_pred_proba, target):
#  print(i, j)
  
